In [ ]:
# incl ctrls in training
mode = 'train'
import numpy as np
singleRange = [1.]
single = 'bagging_fraction'
param = {'min_data_in_leaf'       : 680,  # .25, .34
         'num_leaves'             : 2,
         'max_bin'                : 31,    # .33, .41
#        'lambda_l2'              : 40,
#        'min_sum_hessian_in_leaf': 1e-3,
         'seed'                   : 77,
         'num_boost_round'        : 10000, 
         'early_stopping_round'   : 200, 
         'objective'              : 'binary', 
         'metric'                 : 'binary_logloss', 
         'n_jobs'                 : -1,
         'force_col_wise'         : True,
         'feature_pre_filter'     : False}
#        'bagging_freq'           : 1,
#        'bagging_fraction'       : .1,
#        'feature_fraction'    : .4
if mode=='sweep' and single in param.keys():
    del param[single]
target = 'nfkb_inhibitor'
NFOLD = 5
CASCADE = True
import os, glob, pickle, time, random, string
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from category_encoders import CountEncoder
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import log_loss
from sklearn.model_selection import StratifiedKFold

if 'kid' in os.getcwd():
    DATA = 'data'
elif 'kaggle' in os.getcwd():
    INP = '/kaggle/input'
    DATA = f'{INP}/lish-moa'
    OUT = '/kaggle/working'
SUFFIX = ''.join(random.choices(string.ascii_lowercase+string.digits, k=10))

In [ ]:
if 'kid' not in os.getcwd() and 'jupyter' not in os.getcwd():
    os.system(f'cp -rp {INP}/lgb300-gpu0/LightGBM/ {OUT}')
    os.chdir(f'{OUT}/LightGBM/python-package')
    !python3 setup.py install --precompile
import lightgbm as lgb
assert lgb.__version__=='3.0.0'

In [ ]:
df = {}
for file in glob.glob(f'{DATA}/*.csv'):
    name = file.split('/')[-1].split('.')[0]
    df[name] = pd.read_csv(file, index_col='sig_id')
#    print(name, df[name].isnull().any().any(), '\n', df[name].columns)

# astype category
def cats(df):
    df['cp_dose'] = df['cp_dose'].apply(lambda x: int(x.replace('D', '')))
    df['cp_dose'] = df['cp_dose'].astype('category')
    df['cp_time'] = df['cp_time'].astype('category')
    df['cp_type'] = df['cp_type'].astype('category')
    return df
df['train_features'] = cats(df['train_features'])
df['test_features'] = cats(df['test_features'])

notctrl = df['test_features']['cp_type']=='trt_cp'

In [ ]:
# target matters
vcount = pd.Series(dtype=float)
for col in df['train_targets_scored'].columns:
    t = df['train_targets_scored'][col].value_counts()
    if len(t)==2:
        vcount[col] = t[1]/(t[0]+t[1])
    elif len(t)==1:
        vcount[col] = 0
    else:
        print('oopsy', vcount)
targets = vcount.sort_values(ascending=False).index

In [ ]:
if mode=='train':
    model, predict, logloss = {}, {}, {}
    for target in targets: 
        dataX = df['train_features'].copy()
        test = df['test_features'].copy() 
        datay = df['train_targets_scored'][target].copy()
        if CASCADE:
            for t in predict.keys():
                dataX[t] = predict[t]['train']
                test[t] = predict[t]['test']
        print(target, dataX.shape, test.shape)
        predict[target] = {'train': pd.DataFrame(),
                           'valid': pd.DataFrame(),
                           'test': pd.DataFrame(index=test.index)}

        for n, (idx_t, idx_v) in enumerate(StratifiedKFold(n_splits=NFOLD, random_state=47, shuffle=True).split(dataX, datay)):
            trainX, trainy = dataX.iloc[idx_t], datay.iloc[idx_t]
            validX, validy = dataX.iloc[idx_v], datay.iloc[idx_v]
            trainSet = lgb.Dataset(trainX, label=trainy) 
            validSet = lgb.Dataset(validX, label=validy)
            evalResult = {}
            model[target] = lgb.train(param, trainSet,
                              valid_sets            = [trainSet, validSet], 
                              evals_result          = evalResult,
                              num_boost_round       = param['num_boost_round'],
                              verbose_eval          = False)
            score = model[target].best_score['valid_1']['binary_logloss']
            print(param)
            print('best iteration', model[target].best_iteration)
            print(target, n, score)
            t = model[target].predict(dataX.iloc[idx_t], num_iteration=model[target].best_iteration)
            tis = pd.DataFrame(t, index=dataX.iloc[idx_t].index)
            predict[target]['train'] = pd.concat([predict[target]['train'], tis], axis=1)
            
            t = model[target].predict(dataX.iloc[idx_v], num_iteration=model[target].best_iteration)
            tis = pd.DataFrame(t, index=dataX.iloc[idx_v].index)
            predict[target]['valid'] = pd.concat([predict[target]['valid'], tis])
            
            t = model[target].predict(test, num_iteration=model[target].best_iteration)
            tis = pd.DataFrame(t, index=test.index)
            predict[target]['test'] = pd.concat([predict[target]['test'], tis], axis=1)
        print(len(idx_t), len(idx_v), len(test))
        for k in predict[target].keys():
            print(k, predict[target][k].shape)
        predict[target]['train'] = predict[target]['train'].mean(axis=1)
        predict[target]['test'] = predict[target]['test'].mean(axis=1)
        df['sample_submission'][target] = predict[target]['test']
        logloss[target] = log_loss(datay, predict[target]['valid'])
    with open(f'{OUT}/predict_model.pkl', 'wb') as fp:
        pickle.dump([predict, model], fp)
    df['sample_submission'] = df['sample_submission'].applymap(lambda x: max(min(x, 1-1e-15), 1e-15))
    df['sample_submission'].loc[~notctrl] = 0
    df['sample_submission'].to_csv(f'{OUT}/submission.csv')
    print(logloss, pd.Series(logloss).mean(), df['sample_submission'].describe())
else:
    dataX = df['train_features']
    datay = df['train_targets_scored'][target]
    trainSet = lgb.Dataset(dataX, datay)
    if mode=='imp':
        splits, gains = pd.DataFrame(), pd.DataFrame()
        n = 0
        del param['early_stopping_round']
        print(param)
        while time.time()-tic <= HOUR*3600:
            ttic = time.time()
            param['seed'] = np.random.randint(10000000)
            model = lgb.train(param, trainSet,
                              num_boost_round   = param['num_boost_round'],
                              verbose_eval      = False)
            split = pd.DataFrame(model.feature_importance('split'), index=trainSet.feature_name, columns=[str(n)])
            gain = pd.DataFrame(model.feature_importance('gain'), index=trainSet.feature_name, columns=[str(n)])
            splits = pd.concat([splits, split], axis=1)
            gains = pd.concat([gains, gain], axis=1)
    #       print(gains.shape, gain.iloc[0], time.time()-ttic)
            with open(f'{OUT}/imp_{SUFFIX}.pkl', 'wb') as fp:
                pickle.dump([splits, gains], fp)
            n += 1
        for i, v in gains.mean(axis=1).sort_values(ascending=False).iteritems():
            print('{:30s}{:10.0f}'.format(i, v))
    if mode=='sweep':
        nsweep = 0
        sweeps = pd.DataFrame()
        sweeps.index.name = 'nsweep'
        for value in singleRange:
            print(single, '=', value)
            param.update({single: value}) 
            tic = time.time()  # seconds
            evalHist = lgb.cv(param, trainSet, 
#                             nfold             = 10,
#                             seed              = 77,
                              stratified        = True,
                              eval_train_metric = True)   
            sweep = { 'train_err' : evalHist['train binary_logloss-mean'][-1],
                      'valid_err' : evalHist['valid binary_logloss-mean'][-1],
                      'early_stop': len(evalHist['valid binary_logloss-mean']),
                      'time'      : time.time()-tic}
            sweep.update(param)
            print(sweep)
            sweeps = pd.concat([sweeps, pd.DataFrame(sweep, index=[nsweep])])
            with open(f'{OUT}/sweep_{SUFFIX}.pkl', 'wb') as fp:
                pickle.dump(sweeps, fp)
            nsweep += 1
        idx = abs(sweeps['train_err']-sweeps['valid_err']).idxmin()
        sweeps['train_err'].loc[idx], sweeps['valid_err'].loc[idx]
        param[single] = sweeps.loc[idx, single]
        print('choosing {} = {}'.format(single, param[single]))

In [ ]:
if mode=='sweep':
    for iidx, ddata in sweeps.iterrows():
        print('{:.1f}{:10.3f}{:10.3f}'.format(ddata[single], ddata['train_err'], ddata['valid_err']))

In [ ]:
if 'kid' not in os.getcwd() and 'jupyter' not in os.getcwd():
    !rm -rf {OUT}/LightGBM